<a href="https://colab.research.google.com/github/TheCaveOfAdullam/study2/blob/main/convTEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Dense, Dropout

In [4]:
# Define paths
train_dir = '/content/drive/MyDrive/ship_motor/train'
validation_dir = '/content/drive/MyDrive/ship_motor/validation'
test_dir = '/content/drive/MyDrive/ship_motor/test'

# Set image size and batch size
image_size = (224, 224)
batch_size = 32

# Load datasets
train_dataset = image_dataset_from_directory(train_dir, image_size=image_size, batch_size=batch_size, label_mode='int')
validation_dataset = image_dataset_from_directory(validation_dir, image_size=image_size, batch_size=batch_size, label_mode='int')
test_dataset = image_dataset_from_directory(test_dir, image_size=image_size, batch_size=batch_size, label_mode='int')

# Preprocess datasets
def preprocess_dataset(dataset):
    dataset = dataset.map(lambda x, y: (tf.image.per_image_standardization(x), tf.one_hot(y, depth=4)))
    return dataset

train_dataset = preprocess_dataset(train_dataset)
validation_dataset = preprocess_dataset(validation_dataset)
test_dataset = preprocess_dataset(test_dataset)

# Prefetch datasets for performance
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


Found 7004 files belonging to 4 classes.
Found 1278 files belonging to 4 classes.
Found 1278 files belonging to 4 classes.


In [5]:
# Define the CNN model
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)))
model.add(layers.MaxPool2D((3, 3), strides=(2, 2), padding="same"))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(32, (3, 3), activation="relu"))
model.add(layers.MaxPool2D((3, 3), strides=(2, 2), padding="same"))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(32, (3, 3), activation="relu"))
model.add(layers.MaxPool2D((3, 3), strides=(2, 2), padding="same"))
model.add(layers.BatchNormalization())

model.add(layers.Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))

model.add(Dense(4, activation="softmax"))

# Compile the model
model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

In [6]:
# Train the model
history = model.fit(train_dataset,
                    epochs=10,
                    validation_data=validation_dataset)

Epoch 1/10
219/219 [==============================] - 151s 421ms/step - loss: 0.0368 - accuracy: 0.9936 - val_loss: 7.5555e-09 - val_accuracy: 1.0000
Epoch 2/10
219/219 [==============================] - 81s 367ms/step - loss: 7.3453e-04 - accuracy: 0.9997 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
219/219 [==============================] - 82s 369ms/step - loss: 1.3912e-07 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
219/219 [==============================] - 82s 371ms/step - loss: 2.7075e-06 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
219/219 [==============================] - 81s 364ms/step - loss: 1.3156e-06 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
219/219 [==============================] - 83s 373ms/step - loss: 1.0531e-06 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
219/219 [==============================] - 82s 369ms/step - loss: 2.7226e-

In [7]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {test_accuracy}')

40/40 [==============================] - 11s 197ms/step - loss: 149.9414 - accuracy: 0.7277
Test accuracy: 0.7276995182037354
